In [1]:
%load_ext autoreload
%autoreload 2

# Import dependencies


In [46]:
import os

from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, BaseMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import chain
from langchain_core.utils.function_calling import tool_example_to_messages
from pydantic import BaseModel, Field
from enum import Enum
from langgraph.graph.message import add_messages
from langchain_core.messages import trim_messages
from langchain_google_genai import ChatGoogleGenerativeAI


from IPython.display import display, Markdown, Latex

from typing_extensions import Annotated, TypedDict
from typing import List, Optional, Sequence

# Configure Google credentials

- **NOTE**: Remember change the `GOOGLE_APPLICATION_CREDENTIALS` to the path of your own Google credentials file.


In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/home/cuongdm/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

# Model

In [5]:
model = ChatVertexAI(temperature=0, model="gemini-1.5-flash")

In [6]:
model.invoke([HumanMessage(content="Xin chào, tao là Tứ")])

AIMessage(content='Xin chào Tứ! Rất vui được gặp bạn. 😄 Bạn muốn làm gì hôm nay? \n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.028007585555315018, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.05184546485543251}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.020964214578270912, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.04023800417780876}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.08389048278331757, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.06187598779797554}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.16344575583934784, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLI

In [7]:
model.invoke(
    [HumanMessage(content="Nhiều khi tao muốn một lần nói ra hết tất cả thay vì.")]
)

AIMessage(content='Bạn muốn nói ra hết tất cả thay vì gì? \n\nHãy chia sẻ với tôi những gì bạn đang nghĩ. Tôi ở đây để lắng nghe và hỗ trợ bạn. \n\nCó thể bạn đang muốn:\n\n* **Nói ra những cảm xúc bị kìm nén:** Bạn có thể đang giữ trong lòng những cảm xúc tiêu cực như tức giận, buồn bã, thất vọng, hay sợ hãi. Nói ra chúng có thể giúp bạn giải tỏa và cảm thấy nhẹ nhàng hơn.\n* **Chia sẻ những suy nghĩ và ý tưởng:** Bạn có thể có những ý tưởng hay những suy nghĩ độc đáo mà bạn muốn chia sẻ với người khác. Nói ra chúng có thể giúp bạn nhận được phản hồi và sự hỗ trợ.\n* **Giải quyết một vấn đề:** Bạn có thể đang gặp phải một vấn đề khó khăn và muốn tìm cách giải quyết. Nói ra vấn đề của bạn có thể giúp bạn nhận được lời khuyên và sự giúp đỡ từ người khác.\n\nDù bạn muốn nói ra điều gì, hãy nhớ rằng bạn không cô đơn. Tôi ở đây để lắng nghe và hỗ trợ bạn. \n\nHãy chia sẻ với tôi những gì bạn đang nghĩ, tôi sẽ cố gắng hết sức để giúp bạn. \n', additional_kwargs={}, response_metadata={'is_bl

In [10]:
model.invoke(
    [
        HumanMessage(content="VKS là gì?"),
        AIMessage(
            content="VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng. VKS cung cấp các tính năng như fully managed Kubernetes, autoscaling, monitoring, logging, và nhiều tính năng khác."
        ),
        HumanMessage(content="Các tính năng của VKS là gì?"),
    ]
)

AIMessage(content='VKS (VngCloud Kubernetes Service) cung cấp nhiều tính năng hữu ích cho việc triển khai và quản lý ứng dụng trên Kubernetes, bao gồm:\n\n**Quản lý cụm Kubernetes:**\n\n* **Fully managed Kubernetes:** VKS tự động quản lý và cập nhật các thành phần của cụm Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.\n* **Autoscaling:** Tự động điều chỉnh kích thước cụm Kubernetes dựa trên tải trọng của ứng dụng, đảm bảo hiệu suất và tối ưu hóa chi phí.\n* **Monitoring:** Theo dõi trạng thái và hiệu suất của cụm Kubernetes, giúp bạn phát hiện và giải quyết các vấn đề kịp thời.\n* **Logging:** Thu thập và lưu trữ nhật ký của cụm Kubernetes, giúp bạn phân tích và gỡ lỗi ứng dụng.\n* **Networking:** Cung cấp các dịch vụ mạng cho cụm Kubernetes, bao gồm load balancing, DNS, và ingress controller.\n* **Security:** Bảo mật cụm Kubernetes bằng các tính năng như RBAC, network policy, và container security.\n\n**Phát triển và triển khai ứng dụng:**\n\n* **Deployment:** Triển khai

# Message persistence

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [12]:
# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
config = {"configurable": {"thread_id": "abc123"}}

In [14]:
query = "Làm sao để dạy một con cá biết leo cây?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Bạn không thể dạy một con cá biết leo cây! Cá được thiết kế để sống dưới nước, chúng có mang để thở và vây để bơi. Chúng không có chân, tay, hay khả năng leo trèo. 

Câu hỏi này là một câu đố vui, nhằm nhắc nhở chúng ta rằng mỗi loài động vật đều có những khả năng và giới hạn riêng.


# Async message

In [15]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

In [16]:
# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bạn không thể dạy một con cá biết leo cây! Cá được thiết kế để sống dưới nước, chúng có mang để thở và vây để bơi. Chúng không có chân, tay, hay khả năng leo trèo. 

Câu hỏi này là một câu đố vui, nhằm nhắc nhở chúng ta rằng mỗi loài động vật đều có những khả năng và giới hạn riêng.


# Prompt templates

In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Bạn là một trợ lý ảo. Hãy trả lời các câu hỏi bằng hết khả năng của bạn bằng Tiếng Việt.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [24]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Nếu muốn nói chia tay khi đã hết yêu, làm sao để đối phương không tổn thương? (Vui lòng trả lời bằng tiếng Việt, tiếng anh tao ngu lắm)"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Chia tay là điều không ai muốn, nhất là khi nó khiến người kia tổn thương. Dù bạn đã hết yêu, việc nói lời chia tay vẫn cần sự tế nhị và tôn trọng. Dưới đây là một số cách để giảm thiểu tổn thương cho đối phương:

**1. Chọn thời điểm và địa điểm phù hợp:**

* **Tránh chia tay qua tin nhắn, điện thoại:** Hãy dành thời gian gặp mặt trực tiếp để thể hiện sự tôn trọng và cho phép đối phương được nhìn thấy cảm xúc của bạn.
* **Chọn nơi riêng tư:** Nơi yên tĩnh, không có người xung quanh sẽ giúp cả hai thoải mái chia sẻ cảm xúc.

**2. Nói rõ ràng và chân thành:**

* **Tránh vòng vo, úp mở:** Hãy thẳng thắn nói rằng bạn đã hết yêu và muốn chia tay.
* **Giải thích lý do một cách ngắn gọn:** Không cần phải đi sâu vào chi tiết, chỉ cần giải thích đủ để đối phương hiểu.
* **Thể hiện sự tôn trọng:** Nói lời cảm ơn đối phương vì những gì họ đã dành cho bạn.

**3. Tránh đổ lỗi:**

* **Tập trung vào cảm xúc của bạn:** T

In [26]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hãy tóm gọn nội dung bài hát Em của ngày hôm qua của Sơn Tùng MTP"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bài hát "Em của ngày hôm qua" của Sơn Tùng MTP kể về một mối tình đã qua, một tình yêu đẹp nhưng không thể giữ được. Ca khúc thể hiện nỗi nhớ nhung, tiếc nuối và sự day dứt của người con trai khi phải chia tay người yêu. 

**Nội dung chính:**

* **Nỗi nhớ nhung:** Anh nhớ về những kỷ niệm đẹp của hai người, những lời hứa hẹn, những khoảnh khắc ngọt ngào.
* **Sự tiếc nuối:** Anh tiếc nuối vì đã không giữ được tình yêu, vì đã để em ra đi.
* **Sự day dứt:** Anh day dứt vì những lỗi lầm trong quá khứ, những điều anh đã làm sai khiến em phải rời xa.
* **Sự hi vọng:** Dù biết rằng em đã là quá khứ, nhưng anh vẫn hy vọng một ngày nào đó sẽ gặp lại em.

**Thông điệp:**

Bài hát là lời tự sự của một người đàn ông đang chìm đắm trong nỗi nhớ nhung và tiếc nuối về một tình yêu đã qua. Nó nhắc nhở chúng ta về giá trị của tình yêu và sự quan trọng của việc trân trọng những gì mình đang có.


In [27]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Câu hỏi mà tôi vừa hỏi bạn là gì vậy"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Câu hỏi bạn vừa hỏi tôi là: "Hãy tóm gọn nội dung bài hát Em của ngày hôm qua của Sơn Tùng MTP".


# Using place holder

In [30]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [43]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [44]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Làm sao để nói chia tay nhưng đối phương phải là người chủ động nói"
language = "Vietnamese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Mình hiểu bạn đang muốn chia tay nhưng không muốn là người trực tiếp nói ra. Đây là một tình huống khó xử, nhưng mình có thể đưa ra một số gợi ý:

* **Nói rõ ràng về cảm xúc của bạn:** Hãy chia sẻ với đối phương rằng bạn cảm thấy mối quan hệ không còn phù hợp nữa, bạn không còn hạnh phúc như trước. Hãy tập trung vào cảm xúc của bạn, tránh đổ lỗi cho đối phương.
* **Tạo khoảng cách:** Hãy giảm thiểu thời gian gặp mặt, nhắn tin, gọi điện cho đối phương. Điều này sẽ khiến họ nhận ra sự thay đổi trong mối quan hệ và tự đặt câu hỏi.
* **Nói về tương lai:** Hãy chia sẻ với đối phương về những dự định của bạn trong tương lai, những điều bạn muốn theo đuổi, và cho họ thấy rằng những dự định đó không còn phù hợp với mối quan hệ hiện tại.
* **Để họ tự đưa ra quyết định:** Hãy cho đối phương thời gian để suy nghĩ và đưa ra quyết định. Đừng ép buộc họ phải chia tay, hãy để họ tự nhận ra rằng mối quan hệ này không còn

# Managing Conversation History


In [47]:
trimmer = trim_messages(
    max_tokens=1000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [48]:
messages = [
    SystemMessage(content="Bạn là một trợ lý ảo"),
    HumanMessage(content="Chào mày, tao là Cường"),
    AIMessage(content="Chào bạn."),
    HumanMessage(content="Cho tao biết, VKS là gì vậy?"),
    AIMessage(
        content="VKS là viết tắt của VngCloud Kubernetes Service, một dịch vụ của VngCloud nhằm cung cấp các cụm Kubernetes đến khách hàng. VKS cung cấp các tính năng như fully managed Kubernetes, autoscaling, monitoring, logging, và nhiều tính năng khác."
    ),
    HumanMessage(content="VKS có hỗ trợ CSI không?"),
    AIMessage(
        content="Có chứ, CSI là viết tắt của Container Storage Interface, một chuẩn giao diện cho phép các hệ thống lưu trữ nền tảng cung cấp dịch vụ lưu trữ cho các ứng dụng chạy trên Kubernetes."
    ),
    HumanMessage(content="Làm sao để tạo một PersistentVolumeClaim trên VKS?"),
    AIMessage(
        content="""Đây là một ví dụ để tạo một PersistentVolumeClaim 20Gi trên VKS\n```yaml
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: my-expansion-storage-class                    # [1] The StorageClass name, CAN be changed
provisioner: bs.csi.vngcloud.vn                       # The VNG-CLOUD CSI driver name
parameters:
  type: vtype-61c3fc5b-f4e9-45b4-8957-8aa7b6029018    # The volume type UUID
allowVolumeExpansion: true                            # MUST set this value to turn on volume expansion feature
---

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-expansion-pvc                           # [2] The PVC name, CAN be changed
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi                                # [3] The PVC size, CAN be changed, this value MUST be in the valid range of the proper volume type
  storageClassName: my-expansion-storage-class 
```"""
    ),
    HumanMessage(content="Vậy volume 100Gi thì sao?"),
    AIMessage(content="yes!"),
]

In [53]:
trimmer.invoke(messages)

[SystemMessage(content='Bạn là một trợ lý ảo', additional_kwargs={}, response_metadata={}, id='e695b379-3c4f-431f-a11a-277f331fb496'),
 HumanMessage(content='Vậy volume 100Gi thì sao?', additional_kwargs={}, response_metadata={}, id='91091fd5-68a0-4ebe-b6d2-458b93fd4995'),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, id='f9a888b5-cdb4-4b45-8e67-f008c0cd8206')]

In [58]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # trimmed_messages = trimmer.invoke(state["messages"])
    trimmed_messages = state["messages"]
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [59]:
config = {"configurable": {"thread_id": "abc567"}}
query = "VKS là gì?"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

VKS là viết tắt của **VngCloud Kubernetes Service**, một dịch vụ của VngCloud cung cấp các cụm Kubernetes đến khách hàng. 

Nói cách khác, VKS là một nền tảng quản lý Kubernetes được quản lý đầy đủ (fully managed) bởi VngCloud. Điều này có nghĩa là VngCloud sẽ xử lý tất cả các khía cạnh của việc vận hành và bảo trì cụm Kubernetes, bao gồm:

* **Cài đặt và cấu hình:** VngCloud sẽ cài đặt và cấu hình cụm Kubernetes cho bạn, đảm bảo rằng nó hoạt động trơn tru và an toàn.
* **Quản lý:** VngCloud sẽ quản lý cụm Kubernetes của bạn, bao gồm cập nhật phần mềm, bảo trì hệ thống và xử lý các sự cố.
* **Bảo mật:** VngCloud sẽ đảm bảo rằng cụm Kubernetes của bạn được bảo mật khỏi các mối đe dọa bên ngoài.
* **Khả năng mở rộng:** VngCloud sẽ cho phép bạn dễ dàng mở rộng cụm Kubernetes của bạn khi nhu cầu của bạn thay đổi.

VKS cung cấp một loạt các tính năng, bao gồm:

* **Fully managed Kubernetes:** VngCloud sẽ xử lý

In [60]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Tạo PersistentVolumeClaim 100Gi trên VKS"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Bạn có thể tạo PersistentVolumeClaim 100Gi trên VKS bằng cách sử dụng file YAML sau:

```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 100Gi
  storageClassName: my-storage-class
```

**Giải thích:**

* **apiVersion:** Phiên bản API của Kubernetes.
* **kind:** Loại đối tượng, trong trường hợp này là PersistentVolumeClaim.
* **metadata:** Thông tin meta của PersistentVolumeClaim, bao gồm tên.
* **spec:** Cấu hình của PersistentVolumeClaim.
* **accessModes:** Chế độ truy cập vào PersistentVolumeClaim. Trong trường hợp này là ReadWriteOnce, cho phép một pod duy nhất ghi vào PersistentVolumeClaim.
* **resources:** Yêu cầu tài nguyên cho PersistentVolumeClaim.
    * **requests:** Yêu cầu dung lượng lưu trữ là 100Gi.
* **storageClassName:** Tên của StorageClass được sử dụng để tạo PersistentVolumeClaim.


In [61]:
config = {"configurable": {"thread_id": "abc678"}}
query = "Câu hỏi mà tôi vừa hỏi bạn là gì vậy"
language = "Vietnamese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Câu hỏi bạn vừa hỏi tôi là: "Vậy volume 100Gi thì sao?". 

Bạn muốn hỏi về việc tạo một PersistentVolumeClaim với dung lượng 100Gi trên VKS phải không?


# Streaming

In [63]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Cho tôi xin lời bài hát Em của ngày hôm qua của Sơn Tùng MTP (đoạn rap)"
language = "Vietnamese"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

##| Đoạn rap trong bài hát "Em của ngày hôm qua" của Sơn| Tùng MTP:

**Verse 2:**

(Rap)
|Anh nhớ em, em của ngày hôm qua
Nụ cười, ánh mắt, nụ hôn, lời hứa
Giờ đây đã xa, như| một giấc mơ
Để lại trong anh, nỗi nhớ, niềm đau

Anh nhớ em, em của ngày hôm qua
Tình yêu, hạnh phúc,| giờ đây đã phai nhạt
Chỉ còn lại trong anh, một nỗi cô đơn
Và những kỷ niệm, như những vết thương lòng

Anh nhớ em, em của ngày hôm qua
Nhưng em đã đi, để lại anh| một mình
Trong đêm tối, anh lạc lối, không tìm thấy em
Chỉ còn lại nỗi nhớ, như một lời nguyền

**Lưu ý:** Đây là đoạn rap trong bài hát "Em của ngày hôm qua" của Sơn| Tùng MTP. 
|